#Link Kaggle

In [642]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Imports/Downloads

In [643]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt	
import pandas
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

#Data Preprocessing

In [644]:
df = pd.read_csv('/content/gdrive/MyDrive/io21/sleepdata.csv', header=0, sep=';', quotechar='"')
df = df.drop(columns=['Start', 'End','Sleep Notes','Activity (steps)'])
print(df.shape)
df.head()

(887, 4)


,Sleep quality,Time in bed,Wake up,Heart rate
0,100%,8:32,:),59.0
1,3%,0:16,:|,72.0
2,98%,8:30,:|,57.0
3,65%,7:32,NaN,NaN
4,72%,6:44,:),68.0


In [645]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Sleep quality  887 non-null    object 
 1   Time in bed    887 non-null    object 
 2   Wake up        246 non-null    object 
 3   Heart rate     162 non-null    float64
dtypes: float64(1), object(3)
memory usage: 27.8+ KB


In [646]:
df['Sleep quality'] = df['Sleep quality'].map(lambda x: x.rstrip('%'))

In [647]:
time = pd.DatetimeIndex(df['Time in bed'])
df['Time in bed'] = df['Time in bed'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
df.head()

,Sleep quality,Time in bed,Wake up,Heart rate
0,100,512,:),59.0
1,3,16,:|,72.0
2,98,510,:|,57.0
3,65,452,NaN,NaN
4,72,404,:),68.0


In [648]:
df['Heart rate'].fillna((df['Heart rate'].mean()), inplace=True)
df['Wake up'].fillna('0', inplace=True)

def wakeChange(x):
  if x == ':)':
    return 1
  elif x == ':|':
    return -1
  else:
    return 0

df['Wake up'] = df['Wake up'].apply(wakeChange)

dftrain = df[abs(df['Wake up']) == 1] 
#print(dftrain['Wake up'].values)

export = df[df['Wake up'] == 0] 
export = export.reset_index()
export = export.drop(columns=['Wake up','index'])
export.head()

,Sleep quality,Time in bed,Heart rate
0,65,452,60.604938
1,78,434,60.604938
2,70,325,60.604938
3,78,458,60.604938
4,75,468,60.604938


#Train Model

In [649]:
y = dftrain.pop('Wake up')
X = dftrain
print(y.shape)
print(X.shape)
X_train,X_test,y_train,y_test = train_test_split(X.index,y,test_size=0.2)


(245,)
(245, 3)


In [650]:
model = KNeighborsClassifier(3)
model.fit(X_train.values.reshape(-1,1),y_train)
pred = model.predict(X_test.values.reshape(-1,1))
accuracy = accuracy_score(y_test,pred)
print('accuracy: ', accuracy*100, '%')

accuracy:  87.75510204081633 %


In [651]:
confusion_matrix(y_test, pred, labels = [-1,1])

array([[ 2,  4],
       [ 2, 41]])

#Export

In [652]:
expPred = model.predict(export.values.reshape(-1,1))
print(expPred)
addExp = pd.Series(expPred)
export['pred'] = addExp
export.to_json('data.json')

[ 1 -1  1 ...  1 -1  1]
